In [1]:
import pandas as pd
import requests
import io
import re
import asyncio
from functools import partial
from bs4 import BeautifulSoup
from IPython.core.debugger import set_trace

In [101]:
url_acwi = 'https://www.ishares.com/us/products/239600/ishares-msci-acwi-etf?qt=ACWI'
req = requests.get(url_acwi)

In [7]:
soup = BeautifulSoup(req.text, 'html.parser')

In [95]:
csv_loc = 'https://www.ishares.com/' + soup.find('a', string=re.compile('(?i)detailed holdings'))['href']; csv_loc

'https://www.ishares.com//us/products/239600/ishares-msci-acwi-etf/1467271812596.ajax?fileType=csv&fileName=ACWI_holdings&dataType=fund'

In [99]:
data = requests.get(csv_loc)

In [100]:
cols = '(?i)ticker|name|weight|sector|country'
csv_from = re.search('ticker', data.text, re.IGNORECASE).start()
df = pd.read_csv(io.StringIO(data.text[csv_from:])).filter(regex=cols).dropna()
df

,Ticker,Name,Weight (%),Sector,Country
0,MSFT,MICROSOFT CORP,1.83,Information Technology,United States
1,AAPL,APPLE INC,1.75,Information Technology,United States
2,AMZN,AMAZON COM INC,1.62,Consumer Discretionary,United States
3,INDA,ISHARES MSCI INDIA ETF,1.02,Financials,United States
4,FB,FACEBOOK CLASS A INC,0.83,Communication,United States
5,JPM,JPMORGAN CHASE & CO,0.81,Financials,United States
6,JNJ,JOHNSON & JOHNSON,0.80,Health Care,United States
7,GOOG,ALPHABET INC CLASS C,0.79,Communication,United States
8,GOOGL,ALPHABET INC CLASS A,0.74,Communication,United States
9,XOM,EXXON MOBIL CORP,0.69,Energy,United States


In [8]:
def _name(df):
    name = df.index.name
    if name.lower() != 'region':
        return name
    elif df.index.str.contains('america|asia|europe|africa|middle', case=False).any():
        return name
    else:
        return 'Market Tier'
    
def _df(df, etf):
    return pd.DataFrame({etf.upper():df.dropna().Percentage.str.rstrip('%').astype('float')})

In [20]:
def etf_alloc2(*etfs):
    asset = []
    sector = []
    marketcap = []
    region = []
    markettier = []
    country = []
    
    for etf in etfs:
        _dfs = pd.read_html('https://etfdb.com/etf/'+etf, attrs={'class':'chart base-table'}, index_col=0, flavor=['lxml', 'bs4'])
        dfs = {_name(df):_df(df, etf) for df in _dfs}
        asset.append(dfs['Asset'])
        sector.append(dfs['Sector'])
        marketcap.append(dfs['Market Cap'])
        region.append(dfs['Region'])
        markettier.append(dfs['Market Tier'])
        country.append(dfs['Country'])

    return {
        'asset': pd.concat(asset, axis=1, sort='False').fillna(0), 
        'sector': pd.concat(sector, axis=1, sort='False').fillna(0), 
        'marketcap': pd.concat(marketcap, axis=1, sort='False').fillna(0),
        'region': pd.concat(region, axis=1, sort='False').fillna(0),
        'markettier': pd.concat(markettier, axis=1, sort='False').fillna(0),
        'country': pd.concat(country, axis=1, sort='False').fillna(0)
    }

In [14]:
%%time
allocs = etf_alloc('ACWI','SPY', 'MTUM', 'EEM')

Wall time: 18.1 s


In [11]:
allocs['region']

,ACWI,SPY,MTUM,EEM
Africa,0.76,0.00,0.0,6.12
Asia,7.80,0.00,0.0,67.19
Asia-Pacific,10.85,0.00,0.0,2.46
Europe,21.27,1.88,0.0,9.78
Latin America,1.31,0.00,0.0,11.00
Middle East,0.37,0.00,0.0,2.00
North America,57.63,98.12,100.0,1.46


In [18]:
async get_tables(etf):
    url = 'https://etfdb.com/etf/' + etf
    read_html_partial = partial(pd.read_html, attrs={'class':'chart base-table'}, index_col=0, flavor=['lxml', 'bs4'])
    return await loop.run_in_executor(None, read_html_partial, url)

In [15]:
def etf_alloc(*etfs):
    
    def _name(df):
        name = df.index.name
        if name.lower() != 'region':
            return name
        elif df.index.str.contains('america|asia|europe|africa|middle', case=False).any():
            return name
        else:
            return 'Market Tier'
    
    def _df(df, etf):
        return pd.DataFrame({etf.upper():df.dropna().Percentage.str.rstrip('%').astype('float')})

    
    async def get_tables(etf):
        url = 'https://etfdb.com/etf/' + etf
        read_html_partial = partial(pd.read_html, attrs={'class':'chart base-table'}, index_col=0, flavor=['lxml', 'bs4'])
        tables = await loop.run_in_executor(None, read_html_partial, url)
        return {_name(df):_df(df, etf) for df in tables}


    async def main():
        fts = [asyncio.ensure_future(get_tables(etf)) for etf in etfs]
        return await asyncio.gather(*fts)
    
    asyncio.set_event_loop(asyncio.new_event_loop())
    loop = asyncio.get_event_loop()
    
    # 다음 코드를 주피터에서 돌리려면, tornado를 downgrade 해야함
    # pip install tornado==4.5.3
    res = loop.run_until_complete(main())
    loop.close()
    
    return {
        k:pd.concat([dic[k] for dic in res], axis=1, sort='False').fillna(0) for k in res[0]
    }

In [21]:
%%time
allocs = etf_alloc('ACWI','SPY', 'MTUM', 'EEM')

Wall time: 4.48 s


In [23]:
allocs['Asset']

,ACWI,SPY,MTUM,EEM
Common equity,98.06,99.92,99.69,94.90
ETF Cash Component,1.28,0.08,0.31,1.03
Preferred stock,0.67,0.00,0.00,4.07
